In [25]:
import numpy as np
import pandas as pd
import os

In [69]:
#FASTA file parser code

file="FinalProject.fasta"
f=open(file)
sequence_names=[]
sequences=[]
count=0
seq=""

for line in f:
    if line.startswith(">"):
        sequence_names.append(line[1:].strip()) #to remove the > and \n
        count=count+1
        sequences.append("") #initializing the slot for the sequence
    elif not line.startswith('#') and line!="":
        sequences[count-1]=sequences[count-1]+line.strip().replace('-',"")

f.close()

#initialize data
data = {'ID':sequence_names,
        'Sequence':sequences}
  
#dataframe
dfseq = pd.DataFrame(data)

#print
dfseq

,ID,Sequence
0,HCN1,gccgaggggaggcgctggggcgcgagggctGAGCCGAGCTGAGCCT...
1,HCN2,ccgccccgccccgcccccgcctcccccctCCCTCGGGCTCCGGCCG...
2,KCNA1,gggctgcgcttcgagacgcagctcaagaccCTGGCGCAGTTCCCCA...
3,KCNA2,agcgctgaccaagaaaggaagtggtgatggGGCACATAGAAGAGTG...
4,KCNS3,cctcgctctagcggggcgggaccgacggacAGACCGGCCGACGCGG...
5,KIR3,gcgcggccgcctgtctgcaccggcagcaccATGTCGCTCATGGTCG...


In [75]:
dfseq.Sequence[0]

'...'